# Prediction of Restaurants rating based on review text of Yelp reviews

**To be completed INDIVIDUALLY and due on April 18 at 3pm.**

In [ ]:
print('John Spinelli, U50128653')

** (50 pts) **

In this assignment, we will be working with the [Yelp dataset](http://cs-people.bu.edu/bahargam/506/yelp_dataset_challenge_academic_dataset.tar). You can find the format of the dataset [here](https://www.yelp.com/dataset_challenge).

Our aim is to predict the rating of each restaurant from the reviews **text** of individuals reviews. 

You can use any model you learned in the class and then evaluate your model.

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Apr 13 01:12:16 2017

@author: JohnSpinelli
"""

import pandas as pd
import csv
import numpy as np
import scipy as sp
import re
import json
from pandas import DataFrame as Df
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk import word_tokenize
from sklearn.feature_extraction.text import HashingVectorizer
import sklearn.svm as svm
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

path = "/Users/JohnSpinelli/desktop/yelp_academic_dataset_business.json"
path2 = "/Users/JohnSpinelli/desktop/yelp_academic_dataset_review.json"

### Explain Hashing Vectorizer, and site the source of clean review

def roundRating(ratings):
    return [round(ratings[x] * 2) / 2 for x in range(len(ratings))]

def clean_review(review):
    """
    Function to clean review text to keep only letters and remove stopwords
    Returns a string of the cleaned bill text
    """
    letters_only = re.sub('[^a-zA-Z]', ' ', review)
    words = letters_only.lower().split()
    stopwords_eng = set(stopwords.words("english"))
    useful_words = [x for x in words if not x in stopwords_eng]
    
    # Combine words into a paragraph again
    useful_words_string = ' '.join(useful_words)
    return(useful_words_string)

stemmer = SnowballStemmer("english")

def stem_words(words_list, stemmer):
    return [stemmer.stem(word) for word in words_list]

def tokenize(text):
    tokens = word_tokenize(text)
    stems = stem_words(tokens, stemmer)
    return stems

count = 0
restaurantData = []
categories = []

with open(path,encoding = "ISO-8859-1") as jsonData:
    for line in jsonData:
        if count == 5000:
            break
        
        data_contents = json.loads(line)
        
        if "Restaurants" in data_contents["categories"]:
            restaurantData.append({"business_id":data_contents["business_id"],\
                                   'stars':data_contents['stars']})
            count = count + 1
            

reviews = []
count = 0
with open(path2,encoding = "ISO-8859-1") as jsonData:
    for line in jsonData:
        
        
        line = json.loads(line)
        reviews.append({'business_id':line['business_id'],'text':line['text']})
        count = count + 1


restaurantDf = Df(restaurantData)
reviewDf = Df(reviews)

mergedData = pd.merge(left=restaurantDf,right=reviewDf,on='business_id',how='inner') 
finalData = mergedData.drop_duplicates("business_id")

finalData['clean_review'] = finalData['text'].apply(clean_review)
finalData['stars'] = np.round(finalData['stars'].values)
finalData['stars'] = finalData['stars'].values.astype(int)
vectorizer = HashingVectorizer(decode_error='ignore', n_features = 2**18,
                               tokenizer = tokenize, non_negative=True)



reviewVec = vectorizer.transform(finalData['clean_review'])
stars = finalData['stars']
X_train, X_test, y_train, y_test = train_test_split(reviewVec, stars, test_size=0.2,random_state=42)

svc = svm.SVC(kernel='linear')
svc.fit(X_train, y_train)
y_pred_test = svc.predict(X_test)
print("Accuracy of SVM test set:", svc.score(X_test, y_test))
print("Accuracy of SVM test set:", svc.score(X_train, y_train))










      
        

        

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-

Accuracy of SVM test set: 0.595
Accuracy of SVM test set: 0.701175293823


The clean_review method as well as the decision to use a hashing vectorizer instead of a tfidvectorizer was inspired from online source code that aimed to solve a similar problem involving yelp reviews. https://beckernick.github.io/yelp-reviews/ 

As you can see from the results the accuracy is fairly good after we clean the reviews by stripping them for non-important words as well as stemming. I believe the choice of the SVM linear classifier is the best choice here and is widely used in many attempts to solve this challenge, an alternative option would have been a Naive Bayes Classifier. 